In [ ]:
from xarray import DataArray

from weldx import Q_, U_, GenericSeries

## Allowed variables (expressions only)

In [ ]:
class AllowedVarsSeries(GenericSeries):
    _allowed_variables = ["a"]

In [ ]:
try:
    AllowedVarsSeries("a*b")
except ValueError as e:
    print(e)

In [ ]:
AllowedVarsSeries("4*a");

## Required variables (expressions only)

In [ ]:
class RequiredVarsSeries(GenericSeries):
    _required_variables = ["c"]

In [ ]:
try:
    RequiredVarsSeries("a*b")
except ValueError as e:
    print(e)

In [ ]:
RequiredVarsSeries("a*c");

## Preprocess coordinates

In [ ]:
def handle_time(instance, time, time_ref):
    from weldx import Time

    time = Time(time, time_ref)

    return dict(t=time.as_quantity())

In [ ]:
class PreprocessCoordsSeries(GenericSeries):
    _evaluation_preprocessor = handle_time

### Discrete

In [ ]:
pcs_discrete = PreprocessCoordsSeries(
    Q_([1, 2, 4], "m"), dims=["t"], coords={"t": Q_([2, 4, 7], "s")}
)

In [ ]:
pcs_result_discrete = pcs_discrete(
    time="2000-01-01 15:00:05", time_ref="2000-01-01 15:00:00"
)
print(f"time: {pcs_result_discrete.data_array.t.data}")
print(f"data: {pcs_result_discrete.data}")

In [ ]:
try:
    pcs_discrete(x=3)
except TypeError:
    print("invalid input")

### Expression

In [ ]:
pcs_expr = PreprocessCoordsSeries("a * t", parameters=dict(a="3m/s"))

In [ ]:
pcs_result_expr = pcs_expr(time="2000-01-01 15:00:05", time_ref="2000-01-01 15:00:00")
print(f"time: {pcs_result_expr.data_array.t.data}")
print(f"data: {pcs_result_expr.data}")

In [ ]:
try:
    pcs_expr(x=3)
except TypeError:
    print("invalid input")

## Required dimensions

In [ ]:
class RequiredDimsSeries(GenericSeries):
    _required_dimensions = ["c"]

###  Discrete

In [ ]:
try:
    RequiredDimsSeries(Q_([[1, 2], [3, 4]], "m"), ["a", "b"])
except ValueError as e:
    print(e)

In [ ]:
RequiredDimsSeries(Q_([[1, 2], [3, 4]], "m"), ["a", "c"]);

### Expression

In [ ]:
try:
    RequiredDimsSeries("a*b")
except ValueError as e:
    print(e)

In [ ]:
RequiredDimsSeries("a*b", dims=dict(b="c"));

In [ ]:
RequiredDimsSeries("a*c");

In [ ]:
RequiredDimsSeries("a*b", parameters=dict(a=(Q_([1, 3], "m"), "c")));

## Required dimension units

In [ ]:
class RequiredDimUnitsSeries(GenericSeries):
    _required_dimension_units = dict(t="s", c="m")

### Discrete

In [ ]:
try:
    RequiredDimUnitsSeries(
        Q_([[1, 2], [3, 4]]),
        dims=["t", "c"],
    )
except ValueError as e:
    print(e)

In [ ]:
try:
    RequiredDimUnitsSeries(
        Q_([[1, 2], [3, 4]]),
        dims=["t", "c"],
        coords=dict(t=Q_([1, 2], "s"), c=Q_([1, 2], "K")),
    )
except ValueError as e:
    print(e)

In [ ]:
RequiredDimUnitsSeries(
    Q_([[1, 2], [3, 4]]),
    dims=["t", "c"],
    coords=dict(t=Q_([1, 2], "s"), c=Q_([1, 2], "m")),
);

### Expression

In [ ]:
try:
    RequiredDimUnitsSeries("a*t + c")
except ValueError as e:
    print(e)

In [ ]:
try:
    RequiredDimUnitsSeries(
        "a*t + c", units=dict(t="K", c="m"), parameters=dict(a="m/K")
    )
except ValueError as e:
    print(e)

In [ ]:
try:
    RequiredDimUnitsSeries(
        "a*t + c", units=dict(t="s"), parameters=dict(a="K/s", c="5K")
    )
except ValueError as e:
    print(e)

In [ ]:
RequiredDimUnitsSeries("a*t + c", units=dict(t="s", c="m"), parameters=dict(a="m/s"));

In [ ]:
RequiredDimUnitsSeries("a*t + c", units=dict(t="s"), parameters=dict(a="m/s", c="m"));

In [ ]:
RequiredDimUnitsSeries("a*t + c", units=dict(a="m/s"), parameters=dict(t="s", c="5m"));

## Required unit dimensionality

In [ ]:
class RequiredUnitDimSeries(GenericSeries):
    _required_unit_dimensionality = U_("m").dimensionality

### Discrete

In [ ]:
try:
    RequiredUnitDimSeries(Q_("2K"))
except ValueError as e:
    print(e)

In [ ]:
RequiredUnitDimSeries(Q_("2m"));

### Expression

In [ ]:
try:
    RequiredUnitDimSeries("a*b", units=dict(a="K"))
except ValueError as e:
    print(e)

In [ ]:
RequiredUnitDimSeries("a*b", units=dict(a="m"));

In [ ]:
coords = DataArray(Q_([1, 2, 3], "s"), dims=["t"]).pint.dequantify().pint.dequantify()
data = DataArray(Q_([1, 2, 3], "m"), dims=["t"], coords={"t": coords})
print(data.coords["t"])
print(coords)